### Análise da quantidade de ideogramas por nível de HSK

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

Primeiro, é feito uma leitura dos arquivos com as palavras de cada HSK

In [2]:
DATA_PATH = "../data/"
#São 6 arquivos separados com as palavras, então juntou-se os 6 arquivos em um único dataframe
NAME = "HSK Official With Definitions 2012 L{hsk_level:0d} freqorder.txt"
DATA_DICT = {i : NAME.format(hsk_level=i) for i in range (1, 7)} 

COLUNAS = ['simplificado', 'tradicional', 'pinyin_num', 'pinyin_carac', 'significado']
df_list = list()
for hsk in DATA_DICT:
    aux = pd.read_csv(DATA_PATH + DATA_DICT[hsk], sep='\t', header=0, names=COLUNAS)
    aux['nivel_hsk'] = hsk
    df_list.append(aux.copy())
df = pd.concat(df_list).reset_index(drop=True)
df

,simplificado,tradicional,pinyin_num,pinyin_carac,significado,nivel_hsk
0,我,我,wo3,wǒ,I; me,1
1,你,你,ni3,nǐ,you (singular),1
2,是,是,shi4,shì,be; is; are; am,1
3,了,了,le5,le,indicates a completed or finished action,1
4,不,不,bu4,bù,no; not,1
...,...,...,...,...,...,...
4990,深情厚谊,深情厚誼,shen1qing2hou4yi4,shēnqínghòuyì,profound friendship,6
4991,武侠,武俠,wu3xia2,wǔxiá,knight-errant; a genre of swordplay martial ar...,6
4992,将就,將就,jiang1jiu5,jiāngjiu,put up with; accept somewhat reluctantly,6
4993,对联,對聯,dui4lian2,duìlián,rhyming couplet; vertical written couplet usua...,6


No trecho de código abaixo foi separado cada ideograma das palavras (palavras podem ter um ou mais ideogramas) e colocado em uma lista

In [3]:
palavras = df[['simplificado','nivel_hsk']]

ideo = list()
for i in range(0, palavras['simplificado'].str.len().max()):
    ideo.append(palavras['simplificado'].str[i].dropna())
ideo

lista = pd.Series()
for i in ideo:
    lista = lista.append(i)
lista

<ipython-input-3-9ab62db2a0fc>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lista = pd.Series()


0       我
1       你
2       是
3       了
4       不
       ..
4982    见
4983    深
4987    桑
4988    兽
4990    谊
Length: 9656, dtype: object

In [4]:
palavras = df[['simplificado','nivel_hsk']]

lista = list()
for i in range(0, palavras['simplificado'].str.len().max()):
    lista.append(palavras['simplificado'].str[i].dropna())
lista

ideo = pd.concat(lista).reset_index(drop=True)
ideo

0       我
1       你
2       是
3       了
4       不
       ..
9651    见
9652    深
9653    桑
9654    兽
9655    谊
Name: simplificado, Length: 9656, dtype: object

É possível perceber abaixo que há ideogramas repetidos, isso se dá porque existem palavras que compartilham o mesmo ideograma

In [5]:
ideo[ideo == '一'].count()

31

In [6]:
ideo = ideo.reset_index(drop=True)

Aqui e contado a quantidade de vezes que tal ideograma aparece e ordenado em ordem decrescente   
Também vale destacar que há 2660 ideogramas diferentes em todos os níveis de HSK

In [7]:
df_qtd = pd.DataFrame()
df_qtd['caracter'] = ideo
df_qtd = df_qtd.groupby('caracter').size().reset_index(name='qtd')
df_qtd = df_qtd.sort_values('qtd', ascending=False)
df_qtd

,caracter,qtd
8,不,64
659,子,58
875,心,41
1388,气,39
373,发,38
...,...,...
869,徘,1
870,徙,1
871,御,1
1768,磕,1


Salvando o dataframe acima com os ideogramas de todos os HSK

In [8]:
df_qtd.to_csv('../data/ideogramas por quantidade.txt', index=False)

Aqui pretende-se contar quantas vezes cada ideograma aparece em cada hsk

In [31]:
df_list = list()
aux = pd.DataFrame()
palavras = df[['simplificado','nivel_hsk']]

for i in range(0, 4):
    aux['ideograma'] = palavras['simplificado'].str[i]
    aux['hsk'] = (palavras['nivel_hsk'])
    df_list.append(aux.copy())
ideo = pd.concat(df_list)
ideo = ideo.reset_index(drop=True).dropna()
ideo

,ideograma,hsk
0,我,1
1,你,1
2,是,1
3,了,1
4,不,1
...,...,...
19967,见,6
19968,深,6
19972,桑,6
19973,兽,6


Abaixo foi contado os ideogramas para cada hsk diferente

In [22]:
ideo_hsk = ideo.groupby(['ideograma', 'hsk']).size().reset_index(name='qtd')
ideo_hsk

,ideograma,hsk,qtd
0,一,1,2
1,一,2,3
2,一,3,7
3,一,4,1
4,一,5,8
...,...,...,...
4771,齐,6,2
4772,齿,5,1
4773,龄,4,1
4774,龙,5,1


Para uma melhor visualização, foi pivotada a tabela acima

In [41]:
ideo_pivotado = pd.pivot_table(ideo_hsk, values='qtd', columns='hsk', index='ideograma').fillna(0)
ideo_pivotado[[1,2,3,4,5,6]] = ideo_pivotado[[1,2,3,4,5,6]].astype(int)

ideo_pivotado['total'] = ideo_pivotado.sum(axis=1)
ideo_pivotado = ideo_pivotado.sort_values(['total', 1, 2, 3, 4, 5, 6], ascending=False)
ideo_pivotado

hsk,1,2,3,4,5,6,total
ideograma,,,,,,,
不,3,0,1,8,16,36,64
子,4,2,10,11,20,11,58
心,0,0,4,6,13,18,41
气,2,0,1,4,6,26,39
发,0,0,4,5,10,19,38
...,...,...,...,...,...,...,...
鲁,0,0,0,0,0,1,1
鸣,0,0,0,0,0,1,1
鸦,0,0,0,0,0,1,1


Aqui é possível mudar o valor da variável "hsk" para visulizar os ideogramas mais frequentes no hsk escolhido

In [42]:
hsk = 3
ideo_pivotado.sort_values(hsk, ascending=False).head(20)

hsk,1,2,3,4,5,6,total
ideograma,,,,,,,
子,4,2,10,11,20,11,58
一,2,3,7,1,8,10,31
照,0,0,4,2,2,5,13
过,0,1,4,4,4,8,21
地,0,0,4,3,9,8,24
意,0,1,4,5,2,15,27
关,1,0,4,1,3,4,13
要,0,1,4,2,4,7,18
心,0,0,4,6,13,18,41


Salvando em txt e em excel

In [43]:
ideo_pivotado.to_csv('../data/ideogramas por quantidade e hsk.txt')
ideo_pivotado.to_excel('../data/ideogramas por quantidade e hsk.xlsx')